In [ ]:
#Подключаем необходимые библиотеки
import pandas as pd
import numpy as np
#Загрузка данных
train = pd.read_csv(
    'train_dataset_train.csv', 
    sep = ';',
    dtype = {
        'PATIENT_SEX': str, 
        'MKB_CODE': str, 
        'ADRES': str, 
        'VISIT_MONTH_YEAR': str, 
        'AGE_CATEGORY': str, 
        'PATIENT_ID_COUNT': int
    }
)
test = pd.read_csv(
    'test_dataset_test.csv', 
    sep = ';',
    dtype = {
        'PATIENT_SEX': str, 
        'MKB_CODE': str, 
        'ADRES': str, 
        'VISIT_MONTH_YEAR': str, 
        'AGE_CATEGORY': str, 
        'PATIENT_ID_COUNT': int
    }
)
#Слияние
test = test.merge(        
        train[train['VISIT_MONTH_YEAR'].isin(['03.22'])].groupby(
            ['PATIENT_SEX', 'MKB_CODE', 'ADRES', 'AGE_CATEGORY'], as_index = False)['PATIENT_ID_COUNT'].mean(),    
        on = ['PATIENT_SEX', 'MKB_CODE', 'ADRES', 'AGE_CATEGORY'],
        how = 'left'
    ).fillna(1).astype({'PATIENT_ID_COUNT': int})
#Применяем коэффициенты, для улучшения ответа
test['PATIENT_ID_COUNT'] = test['PATIENT_ID_COUNT'] * 1.19

ind = test[(test['PATIENT_ID_COUNT'] > 100) & (test['PATIENT_ID_COUNT'] < 1000)].index
test.loc[ind, 'PATIENT_ID_COUNT'] = test.loc[ind,'PATIENT_ID_COUNT'] - 20

ind = test[(test['PATIENT_ID_COUNT'] > 1000) & (test['PATIENT_ID_COUNT'] < 2000)].index
test.loc[ind, 'PATIENT_ID_COUNT'] = test.loc[ind,'PATIENT_ID_COUNT'] - 115

ind = test[(test['PATIENT_ID_COUNT'] > 2000) & (test['PATIENT_ID_COUNT'] < 3000)].index
test.loc[ind, 'PATIENT_ID_COUNT'] = test.loc[ind,'PATIENT_ID_COUNT'] - 10

ind = test[test['PATIENT_ID_COUNT'] > 3000].index
test.loc[ind, 'PATIENT_ID_COUNT'] = test.loc[ind,'PATIENT_ID_COUNT'] + 215

test['PATIENT_ID_COUNT'] = np.round(test['PATIENT_ID_COUNT'], 0)

#Формирование файла ответа
test['PATIENT_ID_COUNT'] = test['PATIENT_ID_COUNT'].astype(int)
test.to_csv('final.csv', sep = ';', index = None)
test['PATIENT_ID_COUNT'].head()